In [58]:
SELECT TOP 3 * FROM sales ;
SELECT * FROM members ;
SELECT * FROM menu 

(3 rows affected)

(2 rows affected)

(3 rows affected)

Total execution time: 00:00:00.034

customer_id,order_date,product_id
A,2021-01-01,1
A,2021-01-01,2
A,2021-01-07,2


customer_id,join_date
A,2021-01-07
B,2021-01-09


product_id,product_name,price
1,sushi,10
2,curry,15
3,ramen,12


**1\. What is the total amount each customer spent at the restaurant?**

In [12]:
SELECT s.customer_id, SUM(m.price) AS Product_price
FROM sales s 
JOIN menu m 
    ON s.product_id = m.product_id
GROUP BY customer_id


(3 rows affected)

Total execution time: 00:00:00.019

customer_id,Product_price
A,76
B,74
C,36


**2\. How many days has each customer visited the restaurant?**

In [15]:
SELECT customer_id, COUNT(order_date)
FROM sales
GROUP BY customer_id

(3 rows affected)

Total execution time: 00:00:00.020

customer_id,(No column name)
A,6
B,6
C,3


**3\. What was the first item from the menu purchased by each customer?**

In [78]:
SELECT customer_id, order_date, product_name
FROM 
(SELECT s.customer_id, s.order_date, m.product_name,
    ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date) AS row_num
FROM sales s
JOIN menu m 
    ON s.product_id = m.product_id) as subq
WHERE row_num = 1;


(3 rows affected)

Total execution time: 00:00:00.010

customer_id,order_date,product_name
A,2021-01-01,sushi
B,2021-01-01,curry
C,2021-01-01,ramen


**BONUS: LAST ORDER A CUSTOMER PLACED**

In [77]:
SELECT customer_id, order_date, product_name
FROM
    (SELECT s.customer_id, s.order_date, m.product_name,
        ROW_NUMBER() OVER (PARTITION BY customer_id ORDER BY order_date DESC) AS row_num
    FROM sales s
    JOIN menu m 
        ON s.product_id = m.product_id) as subq
WHERE row_num = 1;

(3 rows affected)

Total execution time: 00:00:00.011

customer_id,order_date,product_name
A,2021-01-11,ramen
B,2021-02-01,ramen
C,2021-01-07,ramen


**4\. What is the most purchased item on the menu and how many times was it purchased by all customers?**

In [36]:
SELECT m.product_name, COUNT(s.product_id) AS [Total Purchase]
FROM sales s
JOIN menu m 
    ON s.product_id = m.product_id
GROUP BY m.product_name
GO

(3 rows affected)

Total execution time: 00:00:00.009

product_name,Total Purchase
curry,4
ramen,8
sushi,3


**5\. Which item was the most popular for each customer?**

In [143]:
SELECT customer_id, product_name, [Product Count]
FROM (
    SELECT s.customer_id, m.product_name, COUNT(s.product_id) [Product Count],
        ROW_NUMBER() OVER (PARTITION BY s.customer_id ORDER BY COUNT(s.product_id) DESC) AS row_num
    FROM sales s
    JOIN menu m 
        ON s.product_id = m.product_id
    GROUP BY s.customer_id,m.product_name
) AS subquery
WHERE row_num = 1

(3 rows affected)

Total execution time: 00:00:00.018

customer_id,product_name,Product Count
A,ramen,3
B,sushi,2
C,ramen,3


 **6. Which item was purchased first by the customer after they became a member?**

In [182]:
SELECT customer_id, product_id, product_name, order_date
FROM (
    SELECT s.customer_id, s.product_id, s.order_date, me.product_name,
        ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.order_date) AS row_num
    FROM members m 
    JOIN sales s 
        ON m.customer_id = s.customer_id
    JOIN menu me
        ON s.product_id = me.product_id
    WHERE join_date <= order_date
    ) AS subquery

WHERE row_num = 1

(2 rows affected)

Total execution time: 00:00:00.026

customer_id,product_id,product_name,order_date
A,2,curry,2021-01-07
B,1,sushi,2021-01-11


**Bonus: Last item purchased after becoming a member**

In [175]:
SELECT customer_id, product_id, product_name, order_date
FROM (
    SELECT s.customer_id, s.product_id, s.order_date, me.product_name,
        ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS row_num
    FROM members m 
    JOIN sales s 
        ON m.customer_id = s.customer_id
    JOIN menu me
        ON s.product_id = me.product_id
    WHERE join_date <= order_date
    ) AS subquery

WHERE row_num = 1

(2 rows affected)

Total execution time: 00:00:00.021

customer_id,product_id,product_name,order_date
A,3,ramen,2021-01-11
B,3,ramen,2021-02-01


**7\. Which item was purchased just before the customer became a member?**

In [186]:
SELECT customer_id, product_id, product_name, order_date
FROM (
    SELECT s.customer_id, s.product_id, s.order_date, me.product_name,
        ROW_NUMBER() OVER(PARTITION BY s.customer_id ORDER BY s.order_date DESC) AS row_num
    FROM members m 
    JOIN sales s 
        ON m.customer_id = s.customer_id
    JOIN menu me
        ON s.product_id = me.product_id
    WHERE join_date > order_date
    ) AS subquery

WHERE row_num = 1

(2 rows affected)

Total execution time: 00:00:00.019

customer_id,product_id,product_name,order_date
A,1,sushi,2021-01-01
B,1,sushi,2021-01-04


 **What is the total items and amount spent for each member before they became a member?**

In [201]:
SELECT s.customer_id, COUNT(s.product_id) AS [Total Item], SUM(me.price) AS [Amount Spent]
FROM sales s
JOIN members m 
    ON s.customer_id = m.customer_id
JOIN menu me 
    ON s.product_id = me.product_id
WHERE m.join_date > s.order_date
GROUP BY s.customer_id;



(2 rows affected)

Total execution time: 00:00:00.016

customer_id,Total Item,Amount Spent
A,2,25
B,3,40


 **9.  If each $1 spent equates to 10 points and sushi has a 2x points multiplier - how many points would each customer have?**

In [67]:
-- Efficient Code
SELECT s.customer_id,
    SUM( CASE WHEN product_name = 'sushi' THEN (m.price * 20) ELSE (m.price * 10) END) AS Points
FROM sales s
JOIN menu m
    ON s.product_id = m.product_id
GROUP BY s.customer_id;


-- Redundant Code
SELECT customer_id, SUM( ISNULL( [Sushi Points], 0) + [Other Points]) AS Points
FROM
(SELECT customer_id,
    SUM( CASE WHEN product_name = 'sushi' THEN Points * 2 END) AS [Sushi Points],
    SUM( CASE WHEN product_name <> 'sushi' THEN Points END) AS [Other Points]
FROM
(SELECT s.customer_id, m.product_name, (m.price * 10) AS Points
FROM sales s
JOIN menu m
    ON s.product_id = m.product_id ) AS subquery
GROUP BY customer_id) AS subquery2
GROUP BY customer_id

(3 rows affected)

Warning: Null value is eliminated by an aggregate or other SET operation.

(3 rows affected)

Total execution time: 00:00:00.020

customer_id,Points
A,860
B,940
C,360


customer_id,Points
A,860
B,940
C,360


 **10. In the first week after a customer joins the program (including their join date) they earn 2x points on all items, not just sushi - how many points do customer A and B have at the end of January?**

In [62]:
-- This query computes the sum of all points in January but also multiplies the points in the first week by 2
-- And note I added 6 to the first week instead of 7 as the join date was also included

SELECT s.customer_id,
    SUM(CASE WHEN order_date BETWEEN join_date AND DATEADD(DAY, 6, join_date) THEN (m.price * 20) ELSE (m.price * 10 )END)
FROM sales s
JOIN menu m
    ON s.product_id = m.product_id 
JOIN members me 
    ON s.customer_id = me.customer_id
WHERE MONTH(order_date) < MONTH('2021-02-01')
GROUP BY s.customer_id


(2 rows affected)

Total execution time: 00:00:00.010

customer_id,(No column name)
A,1270
B,720
